In [10]:
!pip install selenium

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
import urllib
import json
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import seaborn as sns
import sys
import time
import networkx as nx
import matplotlib.pyplot as plt
import re
from collections import defaultdict
%matplotlib inline

In [322]:
# brunch data crawling by 셀레니엄
# source reference : http://stackoverflow.com/questions/12519074/scrape-websites-with-infinite-scrolling

# category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g','시사·이슈?q=g','IT_트렌드?q=g','사진·촬영?q=g','',
#                  '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g','글쓰기_코치?q=g','직장인_현실_조언?q=g'
#                  ,'스타트업_경험담?q=g','육아_이야기?q=g','요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
#                  '디자인_스토리?q=g','문화·예술?q=g','건축·설계?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
#                  '멋진_캘리그래피?q=g','사랑·이별?q=g','감성_에세이?q=g']

category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g']
idlist = []

def crawlBrunchLink(dir):
    ## html crawling
    chromedriver = 'C:/selenium/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver)
    #driver.implicitly_wait(30)
    base_url = "https://brunch.co.kr/keyword/{dir}".format(dir=dir)
    verificationErrors = []
    accept_next_alert = True
    delay = 0.5
    driver.get(base_url)
    #driver.find_element_by_link_text("All").click()
    htmlsize = 0
    keep_cnt = 0
    for i in range(1,2): # 스크롤 횟수
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 페이지 무한 스크롤
        time.sleep(1)
        
        if htmlsize == len(driver.page_source):
            keep_cnt += 1
        else :
            keep_cnt = 0
            htmlsize = len(driver.page_source)
        if keep_cnt > 5 :
            break
            
    html_source = driver.page_source
    ## extract follower, following data
    data = html_source.encode('utf-8')

    soup = BeautifulSoup(data, 'html.parser') 
    classes = soup.find_all('a', href=re.compile('/@@')) # /@@ 로 시작하는 href 태그 찾아오기
#     idlist = []
    for c in classes:
        follwing = c.get('href')
        if follwing is None or len(follwing)==0:
            continue
        idlist.append(follwing[2:])

    driver.close()
    return idlist

In [325]:
writer_list = [] # 크롤링 해온 해당글 url 리스트

for dir in category_list:
    idlist =[] # idlist 누적방지
    crawlBrunchLink(dir)
#     print(idlist)
    writer_list.append(idlist)

In [329]:
import requests

data = []
json_data = {}

def def_craw(writer_list,input_catenum):
    global json_data
    global data
    tmp_text = []
    tag_keyword=[]
    
    tag_title,tag_nickname,tag_publish_date,tag_like,tag_share,tag_url,tag_url_plink = None,None,None,None,None,None,None
    for url in (writer_list[input_catenum]):
        
        if tmp_text == []:
            pass
        else :
            json_data['title'] = tag_title.text  
#             json_data['nickname'] = tag_nickname
            json_data['publish_date'] = tag_publish_date
            json_data['keyword'] = tmp_keyword   
            json_data['like'] = tag_like
#             json_data['share'] = tag_share
            json_data['url'] = tag_url
            json_data['url_plink'] = tag_url_plink 
            json_data['text'] = tmp_text


        data.append(json_data)
        
        json_data = {} # 초기화
        tmp_text = [] # 초기화
        tmp_keyword = [] # 초기화
        
        
        # beautifulsoup 
        html = requests.get('https://brunch.co.kr/@{text_url}'.format(text_url=url))
        soup = BeautifulSoup(html.text, 'html.parser')
        tag_text = soup.find_all('p') # 해당페이지의 본문
        tag_title = soup.find('title') # 해당페이지의 Title
#         tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 글쓴이 닉네임
        tag_url = soup.find("meta",property='og:url')['content'] # 본주소
        tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
        tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 본주소
        tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 해당 글의 키워드
        tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}).text # 좋아요 수
#         tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}).text # 공유 수
        
        for text in tag_text:
            tmp_text.append(text.text) # p태그 text
        for keyword in tag_keyword:
            tmp_keyword.append(keyword.text)

In [330]:
def_craw(writer_list,0)

TypeError: 'NoneType' object is not subscriptable

In [300]:
html = requests.get('https://brunch.co.kr/@{text_url}'.format(text_url='@7LuB/12'))
soup = BeautifulSoup(html.text, 'html.parser')
tag_title = soup.find_all('p') # 해당페이지의 Title 
tag_titlse = soup.find('title') # 해당페이지의 Title
tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 글쓴이 닉네임
tag_url = soup.find("meta",property='og:url')['content'] # 본주소
tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 본주소
tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 해당 글의 키워드
tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}).text # 좋아요 수
tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}).text # 공유 수
# tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수는 이미지임



In [319]:
tag_share

'11'